## Test y generación del dataframe maestro (.parquet)

#### 1. Carga del Df de **Constant (2023) US$**

In [1]:
import pandas as pd

# df_og_Constant -> dataframe original de la hoja "Constant (2023) US$"
df_og_Constant = pd.read_excel(
    "../data/SIPRI-Milex-data-1949-2024_2.xlsx",
    sheet_name = "Constant (2023) US$",
    header = 5,
    na_values= ["...", "xxx", ""]
    )
df_og_Constant

,Country,Unnamed: 1,Notes,1949,1950,1951,1952,1953,1954,1955,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,North Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,NaN,§4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12061.301802,12097.301651,11456.741930,10987.573613,11853.728802,11577.302066,10794.20102,10441.084810,18263.967968,20423.216125
4,Libya,NaN,‡§¶16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1254.746073,1573.682777,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Syria,NaN,§,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Türkiye,NaN,‖,NaN,NaN,NaN,NaN,1014.184494,1025.735369,1037.254254,...,10442.047080,12242.601721,13140.212681,16678.568582,17729.790546,15070.378798,15333.86475,16429.033296,19577.831924,21884.408731
190,United Arab Emirates,NaN,§98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,"Yemen, North",NaN,§100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2. Listar regiones y subregiones para el formato *Long Data* del Df final de **Constant (2023) US$**

In [2]:
list_region = ["Africa", "Americas", "Asia & Oceania", "Europe", "Middle East"]
list_subregion = [
    #Africa
    "North Africa", "sub-Saharan Africa",
    #Americas
    "Central America and the Caribbean", "North America", "South America",
    #Asia & Oceania
    "Oceania", "South Asia", "East Asia", "South East Asia", "Central Asia",
    #Europe
    "Central Europe", "Eastern Europe", "Western Europe"
]
df_og_Constant['Region'] = None
df_og_Constant['Subregion'] = None
current_region = None
current_subregion = None

for index, row in df_og_Constant.iterrows():
    #se limpian espacios en blanco
    celda = str(row['Country']).strip()

    #CASO A -> REGION
    if celda in list_region:
        current_region = celda
        current_subregion = None    #se resetea la variable para cambiar el continente
        df_og_Constant.at[index, 'Tipo_Fila'] = 'Region_Header'

    #CASO B -> SUBREGION
    elif celda in list_subregion:
        current_subregion = celda
        df_og_Constant.at[index, 'Tipo_Fila'] = "Subregion_Header"

    #CASO C -> COUNTRY
    else:
        if current_region is not None:
            df_og_Constant.at[index, 'Region'] = current_region
            df_og_Constant.at[index, 'Subregion'] = current_subregion
            df_og_Constant.at[index, 'Tipo_Fila'] = 'Country'
        else:
            #Filas basura
            df_og_Constant.at[index, 'Tipo_Fila'] = 'Junk'

#df_clean_Constant -> dataframe limpio del df original (formato columnas/long data)
df_clean_Constant = df_og_Constant[df_og_Constant['Tipo_Fila'] == 'Country'].copy()
df_clean_Constant = df_clean_Constant.dropna(subset=['Country'])
df_clean_Constant.drop(columns= ['Unnamed: 1', "Notes"], inplace= True)
df_clean_Constant = df_clean_Constant.reset_index(drop= True)
df_clean_Constant

,Country,1949,1950,1951,1952,1953,1954,1955,1956,1957,...,2018,2019,2020,2021,2022,2023,2024,Region,Subregion,Tipo_Fila
0,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10987.573613,11853.728802,11577.302066,10794.201020,10441.084810,18263.967968,20423.216125,Africa,North Africa,Country
1,Libya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1254.746073,1573.682777,NaN,Africa,North Africa,Country
2,Morocco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118.750569,173.113341,...,3965.969779,4049.107114,5190.109540,5341.867567,5308.917029,5184.928400,5318.166148,Africa,North Africa,Country
3,Tunisia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1015.216143,1248.462738,1307.332601,1312.372853,1253.638778,1208.204190,1229.925954,Africa,North Africa,Country
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1816.612300,1659.925298,1453.030284,1238.132783,1240.298951,1247.846345,914.142477,Africa,sub-Saharan Africa,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Syria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
170,Türkiye,NaN,NaN,NaN,NaN,1014.184494,1025.735369,1037.254254,960.357346,892.267805,...,16678.568582,17729.790546,15070.378798,15333.864750,16429.033296,19577.831924,21884.408731,Middle East,None,Country
171,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
172,"Yemen, North",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country


#### 3. Formato final del Df de **Constant (2023) US$** -> *Long Data* (columnas)

In [3]:
#df_master_Constant -> Df maestro con el formato para el .parquet
id_vars = ['Region', 'Subregion', 'Country', 'Tipo_Fila']
df_master_Constant = df_clean_Constant.melt(
    id_vars = id_vars,
    var_name = 'Year',
    value_name = 'Spending_B'
)
df_master_Constant #-> LISTO PARA MERGEAR

,Region,Subregion,Country,Tipo_Fila,Year,Spending_B
0,Africa,North Africa,Algeria,Country,1949,NaN
1,Africa,North Africa,Libya,Country,1949,NaN
2,Africa,North Africa,Morocco,Country,1949,NaN
3,Africa,North Africa,Tunisia,Country,1949,NaN
4,Africa,sub-Saharan Africa,Angola,Country,1949,NaN
...,...,...,...,...,...,...
13219,Middle East,None,Syria,Country,2024,NaN
13220,Middle East,None,Türkiye,Country,2024,21884.408731
13221,Middle East,None,United Arab Emirates,Country,2024,NaN
13222,Middle East,None,"Yemen, North",Country,2024,NaN


#### 4. Carga del Df de **Share of GDP**

In [4]:
#df_og_ShareGDP -> dataframe original de la hoja "Share of GDP"
df_org_ShareGDP = pd.read_excel(
    "../data/SIPRI-Milex-data-1949-2024_2.xlsx",
    sheet_name= "Share of GDP",
    header= 5,
    na_values= ["...", "xxx", ""]
)
df_org_ShareGDP

,Country,Notes,1949,1950,1951,1952,1953,1954,1955,1956,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,North Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,§4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.055507,0.056527,0.053071,0.049299,0.053270,0.058861,0.048900,0.040575,0.073904,0.079715
4,Libya,‡§¶16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.043467,0.052871,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Syria,§,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Türkiye,‖,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.018128,0.020503,0.020501,0.025221,0.026899,0.021956,0.018589,0.016588,0.017199,0.019226
190,United Arab Emirates,§98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,"Yemen, North",§100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 5. Listar regiones y subregiones para el formato *Long Data* del Df final de **Share of GDP**

In [5]:
df_org_ShareGDP['Region'] = None
df_org_ShareGDP['Subregion'] = None
current_region_gdp = None
current_subregion_gdp = None

for index, row in df_org_ShareGDP.iterrows():
    celda = str(row['Country']).strip()

    if celda in list_region:
        current_region_gdp = celda
        current_subregion_gdp = None
        df_org_ShareGDP.at[index, 'Tipo_Fila'] = 'Region_Header'

    elif celda in list_subregion:
        current_subregion_gdp = celda
        df_org_ShareGDP.at[index, 'Tipo_Fila'] = 'Subregion_Header'

    else:
        if current_region_gdp is not None:
            df_org_ShareGDP.at[index, 'Region'] = current_region_gdp
            df_org_ShareGDP.at[index, 'Subregion'] = current_subregion_gdp
            df_org_ShareGDP.at[index, 'Tipo_Fila'] = 'Country'
        else:
            df_org_ShareGDP.at[index, 'Tipo_Fila'] = 'Junk'

df_clean_ShareGDP = df_org_ShareGDP[df_org_ShareGDP['Tipo_Fila'] == 'Country'].copy()
df_clean_ShareGDP = df_clean_ShareGDP.dropna(subset= ['Country'])
df_clean_ShareGDP.drop(columns= ['Notes'], inplace= True)
df_clean_ShareGDP = df_clean_ShareGDP.reset_index(drop= True)
df_clean_ShareGDP

,Country,1949,1950,1951,1952,1953,1954,1955,1956,1957,...,2018,2019,2020,2021,2022,2023,2024,Region,Subregion,Tipo_Fila
0,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.049299,0.053270,0.058861,0.048900,0.040575,0.073904,0.079715,Africa,North Africa,Country
1,Libya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.043467,0.052871,NaN,Africa,North Africa,Country
2,Morocco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011807,0.01759,...,0.029029,0.028658,0.039797,0.037498,0.038134,0.035926,0.035158,Africa,North Africa,Country
3,Tunisia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.019764,0.023833,0.027128,0.026297,0.025712,0.024871,0.025064,Africa,North Africa,Country
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.019558,0.017679,0.018104,0.013932,0.014102,0.014073,0.010005,Africa,sub-Saharan Africa,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Syria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
170,Türkiye,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.025221,0.026899,0.021956,0.018589,0.016588,0.017199,0.019226,Middle East,None,Country
171,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
172,"Yemen, North",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country


#### 6. Formato final del Df de **Share GDP** -> *Long Data* (Columnas)

In [6]:
#df_master_ShareGDP -> Datframe maestro con el formato Long Data
id_vars_shareGDP = ['Country', 'Region', 'Subregion', 'Tipo_Fila']
df_master_ShareGDP = df_clean_ShareGDP.melt(
    id_vars= id_vars_shareGDP,
    var_name= 'Year',
    value_name= 'Share_of_GDP'
)
df_master_ShareGDP

,Country,Region,Subregion,Tipo_Fila,Year,Share_of_GDP
0,Algeria,Africa,North Africa,Country,1949,NaN
1,Libya,Africa,North Africa,Country,1949,NaN
2,Morocco,Africa,North Africa,Country,1949,NaN
3,Tunisia,Africa,North Africa,Country,1949,NaN
4,Angola,Africa,sub-Saharan Africa,Country,1949,NaN
...,...,...,...,...,...,...
13219,Syria,Middle East,None,Country,2024,NaN
13220,Türkiye,Middle East,None,Country,2024,0.019226
13221,United Arab Emirates,Middle East,None,Country,2024,NaN
13222,"Yemen, North",Middle East,None,Country,2024,NaN


#### 7. Calculo de la variable **Growth Rate** en base al Df de **Constant (2023) US$**

In [7]:
import numpy as np

df_master_Constant_GrowthRate = df_master_Constant.sort_values(by= ['Country', 'Year'])
df_master_Constant_GrowthRate['Growth_Rate'] = df_master_Constant_GrowthRate.groupby('Country')['Spending_B'].pct_change()
df_master_Constant_GrowthRate['Growth_Rate'] = df_master_Constant_GrowthRate['Growth_Rate'].replace([np.inf, -np.inf], np.nan)
df_master_Constant_GrowthRate = df_master_Constant_GrowthRate.reset_index(drop = True)
df_master_Constant_GrowthRate

C:\Users\alani\AppData\Local\Temp\ipykernel_7748\34351411.py:4: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_master_Constant_GrowthRate['Growth_Rate'] = df_master_Constant_GrowthRate.groupby('Country')['Spending_B'].pct_change()


,Region,Subregion,Country,Tipo_Fila,Year,Spending_B,Growth_Rate
0,Asia & Oceania,South Asia,Afghanistan,Country,1949,NaN,NaN
1,Asia & Oceania,South Asia,Afghanistan,Country,1950,NaN,NaN
2,Asia & Oceania,South Asia,Afghanistan,Country,1951,NaN,NaN
3,Asia & Oceania,South Asia,Afghanistan,Country,1952,NaN,NaN
4,Asia & Oceania,South Asia,Afghanistan,Country,1953,NaN,NaN
...,...,...,...,...,...,...,...
13219,Africa,sub-Saharan Africa,Zimbabwe,Country,2020,1415.318292,0.320537
13220,Africa,sub-Saharan Africa,Zimbabwe,Country,2021,2590.767395,0.830519
13221,Africa,sub-Saharan Africa,Zimbabwe,Country,2022,2302.462202,-0.111282
13222,Africa,sub-Saharan Africa,Zimbabwe,Country,2023,870.433538,-0.621955


#### 8. Carga del Df **Per Capita**

In [8]:
#df_og_PerCapita -> dataframe original de la variable Per Capita
df_og_PerCapita = pd.read_excel(
    '../data/SIPRI-Milex-data-1949-2024_2.xlsx',
    sheet_name= 'Per capita',
    header= 6,
    na_values= ['...', 'xxx', '']
)
df_og_PerCapita

,Country,Notes,1988,1989,1990,1991,1992,1993,1994,1995,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,North Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,§4,43.257136,34.955030,36.071210,22.002806,40.060155,47.356145,48.383894,43.805400,...,262.745399,252.716656,244.072592,227.609469,240.154778,222.233660,205.240233,202.686217,398.506861,469.089244
3,Libya,‡§¶16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169.224761,216.492334,NaN
4,Morocco,§17,39.403288,41.756812,44.282009,46.696174,49.133201,49.242222,52.900264,54.894201,...,94.985712,95.587884,98.222568,103.692835,103.289770,132.740463,146.254597,134.513601,138.176324,145.587831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Syria,§,321.374529,123.844120,132.637077,226.432587,225.822787,104.937104,116.870208,66.446991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,Türkiye,‖,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,United Arab Emirates,§98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,"Yemen, North",§100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 9. Listar regiones y subregiones para el formato *Long Data* del Df final de **Per Capita**

In [9]:
df_og_PerCapita['Region'] = None
df_og_PerCapita['Subregion'] = None
current_region_PerCapita = None
current_subregion_PerCapita = None

for index, row in df_og_PerCapita.iterrows():
    celda = str(row['Country']).strip()

    if celda in list_region:
        current_region_PerCapita = celda
        current_subregion_PerCapita = None
        df_og_PerCapita.at[index, 'Tipo_Fila'] = 'Region_Header'

    elif celda in list_subregion:
        current_subregion_PerCapita = celda
        df_og_PerCapita.at[index, 'Tipo_Fila'] = 'Subregion_Header'

    else:
        if current_region_PerCapita is not None:
            df_og_PerCapita.at[index,'Region'] = current_region_PerCapita
            df_og_PerCapita.at[index, 'Subregion'] = current_subregion_PerCapita
            df_og_PerCapita.at[index, 'Tipo_Fila'] = 'Country'
        else:
            df_og_PerCapita.at[index, 'Tipo_Fila'] = 'Junk'

df_clean_PerCapita = df_og_PerCapita[df_og_PerCapita['Tipo_Fila'] == 'Country'].copy()
df_clean_PerCapita = df_clean_PerCapita.dropna(subset= ['Country'])
df_clean_PerCapita.drop(columns=['Notes'], inplace= True)
df_clean_PerCapita = df_clean_PerCapita.reset_index(drop = True)
df_clean_PerCapita

,Country,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2018,2019,2020,2021,2022,2023,2024,Region,Subregion,Tipo_Fila
0,Algeria,43.257136,34.955030,36.071210,22.002806,40.060155,47.356145,48.383894,43.805400,50.508798,...,227.609469,240.154778,222.233660,205.240233,202.686217,398.506861,469.089244,Africa,North Africa,Country
1,Libya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,169.224761,216.492334,NaN,Africa,North Africa,Country
2,Morocco,39.403288,41.756812,44.282009,46.696174,49.133201,49.242222,52.900264,54.894201,54.393110,...,103.692835,103.289770,132.740463,146.254597,134.513601,138.176324,145.587831,Africa,North Africa,Country
3,Tunisia,29.543422,29.021950,30.086900,30.963147,33.710825,31.482802,33.214317,37.710597,43.067451,...,72.094566,84.673220,97.021743,104.039802,95.726686,99.350727,107.327064,Africa,North Africa,Country
4,Angola,137.647118,176.388105,153.206750,87.223868,64.965545,140.568635,45.659671,17.361624,11.466652,...,64.478408,46.210886,30.181173,28.881496,46.254988,34.483277,24.711248,Africa,sub-Saharan Africa,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Syria,321.374529,123.844120,132.637077,226.432587,225.822787,104.937104,116.870208,66.446991,67.154072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
170,Türkiye,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
171,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
172,"Yemen, North",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country


#### 10. Formato final del Df de **Per Capita** -> *Long Data* (Columnas)

In [10]:
#df_master_PerCapita -> Dataframe maestro para la variable Per Capita
id_vars_PerCapita = ['Country', 'Region', 'Subregion', 'Tipo_Fila']
df_master_PerCapita = df_clean_PerCapita.melt(
    id_vars= id_vars_PerCapita,
    var_name= 'Year',
    value_name= 'Per_Capita'
)
df_master_PerCapita

,Country,Region,Subregion,Tipo_Fila,Year,Per_Capita
0,Algeria,Africa,North Africa,Country,1988,43.257136
1,Libya,Africa,North Africa,Country,1988,NaN
2,Morocco,Africa,North Africa,Country,1988,39.403288
3,Tunisia,Africa,North Africa,Country,1988,29.543422
4,Angola,Africa,sub-Saharan Africa,Country,1988,137.647118
...,...,...,...,...,...,...
6433,Syria,Middle East,None,Country,2024,NaN
6434,Türkiye,Middle East,None,Country,2024,NaN
6435,United Arab Emirates,Middle East,None,Country,2024,NaN
6436,"Yemen, North",Middle East,None,Country,2024,NaN


#### 11. Carga del Df de **Share of Govt. spending**

In [11]:
#df_og_GovtSpend -> dataframe origina de la hoja Share of Govt. spending
df_og_GovtSpend = pd.read_excel(
    '../data/SIPRI-Milex-data-1949-2024_2.xlsx',
    sheet_name= 'Share of Govt. spending',
    header= 7,
    na_values= ['...', 'xxx', '']
)
df_og_GovtSpend

,Country,Notes,Reporting year,1988,1989,1990,1991,1992,1993,1994,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,North Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,§4,NaN,NaN,NaN,0.057841,0.040441,0.069904,0.070861,0.093684,...,0.135656,0.152039,0.146409,0.135999,0.143587,0.156941,0.15025,0.124128,0.200418,0.213823
4,Libya,‡§¶16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045085,0.054661,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Syria,§,NaN,NaN,NaN,0.242995,0.304268,0.263180,0.246139,0.243359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Türkiye,‖,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,United Arab Emirates,§98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,"Yemen, North",§100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 12. Listar Regiones y SUbregiones para el formato *Long Data¨* del Df final de **Share of Govt. spending**

In [12]:
df_og_GovtSpend['Region'] = None
df_og_GovtSpend['Subregion'] = None
current_region_GovtSpend = None
current_subregion_GovtSpend = None

for index, row in df_og_GovtSpend.iterrows():
    celda = str(row['Country']).strip()

    if celda in list_region:
        current_region_GovtSpend = celda
        current_subregion_GovtSpend = None
        df_og_GovtSpend.at[index, 'Tipo_Fila'] = 'Region_Header'
    
    elif celda in list_subregion:
        current_subregion_GovtSpend = celda
        df_og_GovtSpend.at[index, 'Tipo_Fila'] = 'Subregion_Header'

    else:
        if current_region_GovtSpend is not None:
            df_og_GovtSpend.at[index, 'Region'] = current_region_GovtSpend
            df_og_GovtSpend.at[index, 'Subregion'] = current_subregion_GovtSpend
            df_og_GovtSpend.at[index, 'Tipo_Fila'] = 'Country'
        else:
            df_og_GovtSpend.at[index, 'Tipo_Fila'] = 'Junk'

df_clean_GovtSpend = df_og_GovtSpend[df_og_GovtSpend['Tipo_Fila'] == 'Country'].copy()
df_clean_GovtSpend = df_clean_GovtSpend.dropna(subset= ['Country'])
df_clean_GovtSpend.drop(columns= ['Notes', 'Reporting year'], inplace= True)
df_clean_GovtSpend = df_clean_GovtSpend.reset_index(drop= True)
df_clean_GovtSpend

,Country,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2018,2019,2020,2021,2022,2023,2024,Region,Subregion,Tipo_Fila
0,Algeria,NaN,NaN,0.057841,0.040441,0.069904,0.070861,0.093684,0.093543,0.106039,...,0.135999,0.143587,0.156941,0.150250,0.124128,0.200418,0.213823,Africa,North Africa,Country
1,Libya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.045085,0.054661,NaN,Africa,North Africa,Country
2,Morocco,NaN,NaN,0.145002,0.158037,0.151130,0.158109,0.162983,0.160040,0.176140,...,0.104896,0.104736,0.116552,0.120053,0.111949,0.109642,0.106734,Africa,North Africa,Country
3,Tunisia,NaN,NaN,NaN,0.063269,0.063291,0.061916,0.064018,0.062582,0.066741,...,0.068902,0.080702,0.078555,0.079197,0.070461,0.070012,0.072678,Africa,North Africa,Country
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060329,...,0.095039,0.085236,0.074624,0.066798,0.058413,0.059104,0.049180,Africa,sub-Saharan Africa,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Syria,NaN,NaN,0.242995,0.304268,0.263180,0.246139,0.243359,0.233547,0.218170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
170,Türkiye,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
171,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country
172,"Yemen, North",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Middle East,None,Country


#### 13. Formato final del Df de **Share of Govt. spending** -> *Long Data* (Columnas)

In [13]:
#df_master_GovtSpend -> Dataframe maestro de la variable Share of Govt. spending
id_vars_GovtSpend = ['Country', 'Region', 'Subregion', 'Tipo_Fila']
df_master_GovtSpend = df_clean_GovtSpend.melt(
    id_vars= id_vars_GovtSpend,
    var_name= 'Year',
    value_name= 'Share_of_Govt_Spending'
)
df_master_GovtSpend

,Country,Region,Subregion,Tipo_Fila,Year,Share_of_Govt_Spending
0,Algeria,Africa,North Africa,Country,1988,NaN
1,Libya,Africa,North Africa,Country,1988,NaN
2,Morocco,Africa,North Africa,Country,1988,NaN
3,Tunisia,Africa,North Africa,Country,1988,NaN
4,Angola,Africa,sub-Saharan Africa,Country,1988,NaN
...,...,...,...,...,...,...
6433,Syria,Middle East,None,Country,2024,NaN
6434,Türkiye,Middle East,None,Country,2024,NaN
6435,United Arab Emirates,Middle East,None,Country,2024,NaN
6436,"Yemen, North",Middle East,None,Country,2024,NaN


#### 14. Creacion del Df **Final** con todos los dataframes mergeados

In [14]:
#df_final -> Dataframe final en donde se hace el Merge de todas los demas Df
df_final = df_master_Constant_GrowthRate.copy() #-> Variables: Spending_B y Growth_Rate
print(f"Filas iniciales: {len(df_final)}") #elemento de control

#Merge - Variables: Spending_B, Growth_Rate, Share_of_GDP (NUEVA)
df_final = pd.merge(
    df_final,
    df_master_ShareGDP[['Country', 'Year', 'Share_of_GDP']],
    on= ['Country', 'Year'],
    how= 'left'
)
print(f"Filas tras añadir PBI {len(df_final)}")

#Merge - Variables: Spending_B, Growth_Rate, Share_of_GDP, Per_Capita (NUEVA)
df_final = pd.merge(
    df_final,
    df_master_PerCapita[['Country', 'Year', 'Per_Capita']],
    on= ['Country', 'Year'],
    how= 'left'
)
print(f"Filas tras añadir Per Capita {len(df_final)}")

#Merge - Variables: Spending_B, Growth_Rate, Share_of_GDP, Per_Capita, Share_of_Govt_Spending (NUEVA)
df_final = pd.merge(
    df_final,
    df_master_GovtSpend[['Country','Year','Share_of_Govt_Spending']],
    on= ['Country', 'Year'],
    how= 'left'
)
print(f"Filas tras añadir Share of Govt Spending {len(df_final)}")


Filas iniciales: 13224
Filas tras añadir PBI 13224
Filas tras añadir Per Capita 13224
Filas tras añadir Share of Govt Spending 13224


#### 15. Dataframe *final* con toda la informacion:

In [15]:
df_final

,Region,Subregion,Country,Tipo_Fila,Year,Spending_B,Growth_Rate,Share_of_GDP,Per_Capita,Share_of_Govt_Spending
0,Asia & Oceania,South Asia,Afghanistan,Country,1949,NaN,NaN,NaN,NaN,NaN
1,Asia & Oceania,South Asia,Afghanistan,Country,1950,NaN,NaN,NaN,NaN,NaN
2,Asia & Oceania,South Asia,Afghanistan,Country,1951,NaN,NaN,NaN,NaN,NaN
3,Asia & Oceania,South Asia,Afghanistan,Country,1952,NaN,NaN,NaN,NaN,NaN
4,Asia & Oceania,South Asia,Afghanistan,Country,1953,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13219,Africa,sub-Saharan Africa,Zimbabwe,Country,2020,1415.318292,0.320537,0.009750,15.732077,0.065961
13220,Africa,sub-Saharan Africa,Zimbabwe,Country,2021,2590.767395,0.830519,0.015678,30.264705,0.077056
13221,Africa,sub-Saharan Africa,Zimbabwe,Country,2022,2302.462202,-0.111282,0.009950,20.879657,0.040221
13222,Africa,sub-Saharan Africa,Zimbabwe,Country,2023,870.433538,-0.621955,0.002098,53.720517,0.011680


In [16]:
cortes = [1948, 1991, 2000, 2013, 2021, 2025]

nombre_eras = [
    "1. Guerra Fría (1949-1991)",
    "2. Posguerra Fría (1992-2000)",
    "3. Guerra contra el Terror (2001-2013)",
    "4. Resurgimiento de Tensiones (2014-2022)",
    "5. Inestabilidad Global (2022-Presente)"
]

df_final['Historical_Era'] = pd.cut(
    df_final['Year'],
    bins= cortes,
    labels= nombre_eras
)
df_final

,Region,Subregion,Country,Tipo_Fila,Year,Spending_B,Growth_Rate,Share_of_GDP,Per_Capita,Share_of_Govt_Spending,Historical_Era
0,Asia & Oceania,South Asia,Afghanistan,Country,1949,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
1,Asia & Oceania,South Asia,Afghanistan,Country,1950,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
2,Asia & Oceania,South Asia,Afghanistan,Country,1951,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
3,Asia & Oceania,South Asia,Afghanistan,Country,1952,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
4,Asia & Oceania,South Asia,Afghanistan,Country,1953,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
...,...,...,...,...,...,...,...,...,...,...,...
13219,Africa,sub-Saharan Africa,Zimbabwe,Country,2020,1415.318292,0.320537,0.009750,15.732077,0.065961,4. Resurgimiento de Tensiones (2014-2022)
13220,Africa,sub-Saharan Africa,Zimbabwe,Country,2021,2590.767395,0.830519,0.015678,30.264705,0.077056,4. Resurgimiento de Tensiones (2014-2022)
13221,Africa,sub-Saharan Africa,Zimbabwe,Country,2022,2302.462202,-0.111282,0.009950,20.879657,0.040221,5. Inestabilidad Global (2022-Presente)
13222,Africa,sub-Saharan Africa,Zimbabwe,Country,2023,870.433538,-0.621955,0.002098,53.720517,0.011680,5. Inestabilidad Global (2022-Presente)


In [23]:
%pip install pyarrow fastparquet

  Using cached pyarrow-22.0.0-cp313-cp313-win_amd64.whl.metadata (3.3 kB)
  Using cached fastparquet-2025.12.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached cramjam-2.11.0-cp313-cp313-win_amd64.whl.metadata (681 bytes)
  Using cached fsspec-2026.1.0-py3-none-any.whl.metadata (10 kB)
Using cached pyarrow-22.0.0-cp313-cp313-win_amd64.whl (28.0 MB)
Using cached fastparquet-2025.12.0-cp313-cp313-win_amd64.whl (667 kB)
Using cached cramjam-2.11.0-cp313-cp313-win_amd64.whl (1.7 MB)
Using cached fsspec-2026.1.0-py3-none-any.whl (201 kB)

   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 0/4 [pyarrow]
   ---------------------------------------- 

In [17]:
df_final.to_parquet("../data/datos_militares_final.parquet", index= False)

In [1]:
import pandas as pd

df = pd.read_parquet("../data/datos_militares_final.parquet")
df

,Region,Subregion,Country,Tipo_Fila,Year,Spending_B,Growth_Rate,Share_of_GDP,Per_Capita,Share_of_Govt_Spending,Historical_Era
0,Asia & Oceania,South Asia,Afghanistan,Country,1949,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
1,Asia & Oceania,South Asia,Afghanistan,Country,1950,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
2,Asia & Oceania,South Asia,Afghanistan,Country,1951,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
3,Asia & Oceania,South Asia,Afghanistan,Country,1952,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
4,Asia & Oceania,South Asia,Afghanistan,Country,1953,NaN,NaN,NaN,NaN,NaN,1. Guerra Fría (1949-1991)
...,...,...,...,...,...,...,...,...,...,...,...
13219,Africa,sub-Saharan Africa,Zimbabwe,Country,2020,1415.318292,0.320537,0.009750,15.732077,0.065961,4. Resurgimiento de Tensiones (2014-2022)
13220,Africa,sub-Saharan Africa,Zimbabwe,Country,2021,2590.767395,0.830519,0.015678,30.264705,0.077056,4. Resurgimiento de Tensiones (2014-2022)
13221,Africa,sub-Saharan Africa,Zimbabwe,Country,2022,2302.462202,-0.111282,0.009950,20.879657,0.040221,5. Inestabilidad Global (2022-Presente)
13222,Africa,sub-Saharan Africa,Zimbabwe,Country,2023,870.433538,-0.621955,0.002098,53.720517,0.011680,5. Inestabilidad Global (2022-Presente)
